In [1]:
# tylko nowe cechy
# liczba nowych cech to połowa oryginalnych cech
# sqrt(n) zaokrąglony w góre
# klasyfikatory - zafiksowac sensowne ustawienia
# wagi sumowane do 1
# lecim po wszystkich zbiorach czemu nie
# k-fold cross validation
from multi_imbalance.datasets import load_datasets
import experiment as ex
from sklearn.model_selection import StratifiedKFold
from collections import Counter, defaultdict
from multi_imbalance.utils.metrics import gmean_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pandas as pd

/home/dhorna/anaconda3/envs/mgr/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.datasets.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
datasets = load_datasets()

In [3]:
config = ex.config_calculation_strategy2(datasets)

In [5]:
config

{'1czysty-cut': {'nn_config': {'units_1st_layer': 2,
   'units_2nd_layer': 16,
   'units_3rd_layer': 8,
   'units_latent_layer': 4},
  'weighted_triplet_loss': True},
 '2delikatne-cut': {'nn_config': {'units_1st_layer': 2,
   'units_2nd_layer': 16,
   'units_3rd_layer': 8,
   'units_latent_layer': 4},
  'weighted_triplet_loss': True},
 '3mocniej-cut': {'nn_config': {'units_1st_layer': 2,
   'units_2nd_layer': 16,
   'units_3rd_layer': 8,
   'units_latent_layer': 4},
  'weighted_triplet_loss': True},
 '4delikatne-bezover-cut': {'nn_config': {'units_1st_layer': 2,
   'units_2nd_layer': 16,
   'units_3rd_layer': 8,
   'units_latent_layer': 4},
  'weighted_triplet_loss': True},
 'balance-scale': {'nn_config': {'units_1st_layer': 4,
   'units_2nd_layer': 16,
   'units_3rd_layer': 8,
   'units_latent_layer': 4},
  'weighted_triplet_loss': True},
 'cleveland': {'nn_config': {'units_1st_layer': 13,
   'units_2nd_layer': 39,
   'units_3rd_layer': 26,
   'units_latent_layer': 6},
  'weighted_tri

In [6]:
def instantiate_classifiers():
    classifiers = {
            "RF24": RandomForestClassifier(random_state=0, min_samples_leaf=2, min_samples_split=4),
            "RF4": RandomForestClassifier(random_state=0, max_depth=4),
            "RF5": RandomForestClassifier(random_state=0, max_depth=5),
            "RF7": RandomForestClassifier(random_state=0, max_depth=7)
        } 
    return classifiers

In [7]:
result_dict = defaultdict(list)

# for dataset_name in list(datasets.keys())[5:6]:
for dataset_name in list(datasets.keys()):
    print(dataset_name)
    result_dict["dataset"].append(dataset_name)
    X, y = datasets[dataset_name]['data'], datasets[dataset_name]['target']
    skf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
    n_splits = skf.get_n_splits(X, y)
    fold_result_dict = defaultdict(list)
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        weights = ex.weights_calculation_strategy1(X_train, y_train)
        nn_config =  config[dataset_name]['nn_config']
        # pretrain ANN triplets
        embeddings_train, embeddings_test = ex.train_triplets(X_train, y_train, X_test, y_test, weights,nn_config)
        X_train_new = np.hstack([X_train, embeddings_train])
        X_test_new = np.hstack([X_test, embeddings_test])
        
        classifiers = instantiate_classifiers()
        for clf_name, clf in classifiers.items():
            clf.fit(X_train, y_train)
            preds = clf.predict(X_test)

            gmean = gmean_score(y_test, preds)
            f1 = f1_score(y_test, preds, average='macro')
            fold_result_dict[f"{clf_name}-f1-old-features"].append(f1)
            fold_result_dict[f"{clf_name}-gmean-old-features"].append(gmean)
            
        classifiers = instantiate_classifiers()
        for clf_name, clf in classifiers.items():
            clf.fit(X_train_new, y_train)
            preds = clf.predict(X_test_new)

            gmean = gmean_score(y_test, preds)
            f1 = f1_score(y_test, preds, average='macro')
            fold_result_dict[f"{clf_name}-f1-old+new-features"].append(f1)
            fold_result_dict[f"{clf_name}-gmean-old+new-features"].append(gmean)
            
        classifiers = instantiate_classifiers()
        for clf_name, clf in classifiers.items():
            clf.fit(embeddings_train, y_train)
            preds = clf.predict(embeddings_test)

            gmean = gmean_score(y_test, preds)
            f1 = f1_score(y_test, preds, average='macro')
            fold_result_dict[f"{clf_name}-f1-new-features"].append(f1)
            fold_result_dict[f"{clf_name}-gmean-new-features"].append(gmean)
    for k in fold_result_dict:
        result_dict[k].append(np.mean(fold_result_dict[k]))
        
            
            #f1, gmean
    # mean over folds
    # wiersze: datasety, 
    # kolumny -> klasyfikatory (przed dodaniem cech, po dodaniu, na samych nowych cechach)

1czysty-cut
Train Epoch: 1 [0/960 (0%)]	Loss: 45.303040
Train Epoch: 1 [60/960 (67%)]	Loss: 8.847566

Test set: Average loss: 63.2584

Train Epoch: 2 [0/960 (0%)]	Loss: 4.885962
Train Epoch: 2 [60/960 (67%)]	Loss: 4.794541

Test set: Average loss: 19.5493

Train Epoch: 3 [0/960 (0%)]	Loss: 3.354056
Train Epoch: 3 [60/960 (67%)]	Loss: 1.567950

Test set: Average loss: 12.8072

Train Epoch: 4 [0/960 (0%)]	Loss: 2.540012
Train Epoch: 4 [60/960 (67%)]	Loss: 1.010205

Test set: Average loss: 9.3397

Train Epoch: 5 [0/960 (0%)]	Loss: 0.538487
Train Epoch: 5 [60/960 (67%)]	Loss: 1.384090

Test set: Average loss: 8.5082

Train Epoch: 6 [0/960 (0%)]	Loss: 0.914052
Train Epoch: 6 [60/960 (67%)]	Loss: 0.448879

Test set: Average loss: 6.9913

Train Epoch: 7 [0/960 (0%)]	Loss: 0.829111
Train Epoch: 7 [60/960 (67%)]	Loss: 0.631709

Test set: Average loss: 6.7824

Train Epoch: 8 [0/960 (0%)]	Loss: 0.880200
Train Epoch: 8 [60/960 (67%)]	Loss: 0.293910

Test set: Average loss: 6.2480

Train Epoch: 9 [

 34%|███▍      | 326/960 [00:00<00:00, 3228.74it/s]


Test set: Average loss: 2.6261



100%|██████████| 240/240 [00:00<00:00, 3294.40it/s]


Train Epoch: 1 [0/960 (0%)]	Loss: 74.227791
Train Epoch: 1 [60/960 (67%)]	Loss: 9.033646

Test set: Average loss: 66.6140

Train Epoch: 2 [0/960 (0%)]	Loss: 6.685593
Train Epoch: 2 [60/960 (67%)]	Loss: 5.111343

Test set: Average loss: 19.4060

Train Epoch: 3 [0/960 (0%)]	Loss: 3.176573
Train Epoch: 3 [60/960 (67%)]	Loss: 2.419269

Test set: Average loss: 10.9042

Train Epoch: 4 [0/960 (0%)]	Loss: 1.444167
Train Epoch: 4 [60/960 (67%)]	Loss: 1.561696

Test set: Average loss: 8.5013

Train Epoch: 5 [0/960 (0%)]	Loss: 1.610001
Train Epoch: 5 [60/960 (67%)]	Loss: 1.199099

Test set: Average loss: 6.8847

Train Epoch: 6 [0/960 (0%)]	Loss: 1.149880
Train Epoch: 6 [60/960 (67%)]	Loss: 1.101485

Test set: Average loss: 5.9821

Train Epoch: 7 [0/960 (0%)]	Loss: 0.867504
Train Epoch: 7 [60/960 (67%)]	Loss: 1.037183

Test set: Average loss: 5.2462

Train Epoch: 8 [0/960 (0%)]	Loss: 1.309154
Train Epoch: 8 [60/960 (67%)]	Loss: 1.036532

Test set: Average loss: 4.8535

Train Epoch: 9 [0/960 (0%)]	

...

100%|██████████| 43/43 [00:00<00:00, 2612.71it/s]


Train Epoch: 1 [0/172 (0%)]	Loss: 0.186046

Test set: Average loss: 0.3010

Train Epoch: 2 [0/172 (0%)]	Loss: 0.148309

Test set: Average loss: 0.2789

Train Epoch: 3 [0/172 (0%)]	Loss: 0.126137

Test set: Average loss: 0.3648

Train Epoch: 4 [0/172 (0%)]	Loss: 0.075657

Test set: Average loss: 0.2806

Train Epoch: 5 [0/172 (0%)]	Loss: 0.031289

Test set: Average loss: 0.2037

Train Epoch: 6 [0/172 (0%)]	Loss: 0.125345

Test set: Average loss: 0.3082

Train Epoch: 7 [0/172 (0%)]	Loss: 0.030741

Test set: Average loss: 0.2146

Train Epoch: 8 [0/172 (0%)]	Loss: 0.010746

Test set: Average loss: 0.2969

Train Epoch: 9 [0/172 (0%)]	Loss: 0.090671

Test set: Average loss: 0.2404

Train Epoch: 10 [0/172 (0%)]	Loss: 0.029084

Test set: Average loss: 0.1631

Train Epoch: 11 [0/172 (0%)]	Loss: 0.049739

Test set: Average loss: 0.1518

Train Epoch: 12 [0/172 (0%)]	Loss: 0.026249

Test set: Average loss: 0.1996

Train Epoch: 13 [0/172 (0%)]	Loss: 0.033729

Test set: Average loss: 0.1535

Train Ep

  0%|          | 0/43 [00:00<?, ?it/s]


Test set: Average loss: 0.2042



100%|██████████| 43/43 [00:00<00:00, 2642.33it/s]


In [8]:
results_df = pd.DataFrame.from_dict(result_dict)

In [9]:
results_df

,dataset,RF24-f1-old-features,RF24-gmean-old-features,RF4-f1-old-features,RF4-gmean-old-features,RF5-f1-old-features,RF5-gmean-old-features,RF7-f1-old-features,RF7-gmean-old-features,RF24-f1-old+new-features,...,RF7-f1-old+new-features,RF7-gmean-old+new-features,RF24-f1-new-features,RF24-gmean-new-features,RF4-f1-new-features,RF4-gmean-new-features,RF5-f1-new-features,RF5-gmean-new-features,RF7-f1-new-features,RF7-gmean-new-features
0,1czysty-cut,0.927800,0.918606,0.905429,0.896495,0.905533,0.902969,0.928631,0.924891,0.955033,...,0.955776,0.943420,0.915376,0.905639,0.865260,0.845573,0.887043,0.879214,0.904779,0.899292
1,2delikatne-cut,0.785844,0.751689,0.760193,0.696414,0.777951,0.725794,0.786406,0.747480,0.766634,...,0.773095,0.723168,0.733997,0.676100,0.631987,0.459732,0.689131,0.587150,0.718577,0.651873
2,3mocniej-cut,0.573803,0.463827,0.498805,0.082434,0.578375,0.416180,0.584513,0.452692,0.576911,...,0.595548,0.458436,0.513044,0.334937,0.478029,0.079429,0.513390,0.266674,0.512915,0.276751
3,4delikatne-bezover-cut,0.818686,0.804856,0.829701,0.794818,0.827701,0.815232,0.833611,0.828137,0.818612,...,0.824070,0.812660,0.815971,0.798404,0.794294,0.746877,0.813733,0.773361,0.809200,0.792369
4,balance-scale,0.613201,0.096373,0.602859,0.096128,0.609438,0.096721,0.608746,0.095293,0.817630,...,0.834250,0.809652,0.795898,0.773531,0.803586,0.797872,0.806064,0.793688,0.787517,0.756226
5,cleveland,0.316170,0.083242,0.285711,0.073227,0.284363,0.074887,0.296950,0.080578,0.287961,...,0.317990,0.087694,0.269492,0.061507,0.238108,0.037420,0.257429,0.050629,0.257035,0.049438
6,cleveland_v2,0.296382,0.041099,0.269112,0.025991,0.266816,0.025892,0.323781,0.053368,0.325879,...,0.335868,0.056487,0.300931,0.043006,0.280778,0.029524,0.294584,0.030704,0.308302,0.046169
7,cmc,0.522486,0.507424,0.509892,0.479466,0.514592,0.487395,0.541080,0.523951,0.517754,...,0.531180,0.512430,0.474723,0.460930,0.491365,0.475940,0.489945,0.474851,0.493027,0.480539
8,dermatology,0.963844,0.955977,0.966801,0.958125,0.969889,0.961607,0.969889,0.961607,0.959769,...,0.960594,0.956583,0.948944,0.944833,0.949357,0.941425,0.951957,0.947861,0.940354,0.932151
9,glass,0.704754,0.494085,0.638555,0.294955,0.688237,0.419342,0.704035,0.489533,0.632168,...,0.663647,0.375719,0.481733,0.151016,0.473318,0.131206,0.491606,0.133640,0.490274,0.132573


In [10]:
results_df.filter(regex=("dataset|RF24-f1*"))

,dataset,RF24-f1-old-features,RF24-f1-old+new-features,RF24-f1-new-features
0,1czysty-cut,0.927800,0.955033,0.915376
1,2delikatne-cut,0.785844,0.766634,0.733997
2,3mocniej-cut,0.573803,0.576911,0.513044
3,4delikatne-bezover-cut,0.818686,0.818612,0.815971
4,balance-scale,0.613201,0.817630,0.795898
5,cleveland,0.316170,0.287961,0.269492
6,cleveland_v2,0.296382,0.325879,0.300931
7,cmc,0.522486,0.517754,0.474723
8,dermatology,0.963844,0.959769,0.948944
9,glass,0.704754,0.632168,0.481733


In [11]:
results_df.filter(regex=("dataset|RF24-gmean*"))

,dataset,RF24-gmean-old-features,RF24-gmean-old+new-features,RF24-gmean-new-features
0,1czysty-cut,0.918606,0.938336,0.905639
1,2delikatne-cut,0.751689,0.724523,0.676100
2,3mocniej-cut,0.463827,0.465795,0.334937
3,4delikatne-bezover-cut,0.804856,0.807981,0.798404
4,balance-scale,0.096373,0.800727,0.773531
5,cleveland,0.083242,0.057905,0.061507
6,cleveland_v2,0.041099,0.064364,0.043006
7,cmc,0.507424,0.504383,0.460930
8,dermatology,0.955977,0.954153,0.944833
9,glass,0.494085,0.336080,0.151016


In [12]:
results_df.filter(regex=("dataset|RF4-f1*"))

,dataset,RF4-f1-old-features,RF4-f1-old+new-features,RF4-f1-new-features
0,1czysty-cut,0.905429,0.900446,0.865260
1,2delikatne-cut,0.760193,0.720446,0.631987
2,3mocniej-cut,0.498805,0.510885,0.478029
3,4delikatne-bezover-cut,0.829701,0.814423,0.794294
4,balance-scale,0.602859,0.829835,0.803586
5,cleveland,0.285711,0.270233,0.238108
6,cleveland_v2,0.269112,0.284007,0.280778
7,cmc,0.509892,0.526679,0.491365
8,dermatology,0.966801,0.965969,0.949357
9,glass,0.638555,0.563459,0.473318


In [13]:
results_df.filter(regex=("dataset|RF4-gmean*"))

,dataset,RF4-gmean-old-features,RF4-gmean-old+new-features,RF4-gmean-new-features
0,1czysty-cut,0.896495,0.882501,0.845573
1,2delikatne-cut,0.696414,0.623732,0.459732
2,3mocniej-cut,0.082434,0.213308,0.079429
3,4delikatne-bezover-cut,0.794818,0.770187,0.746877
4,balance-scale,0.096128,0.821558,0.797872
5,cleveland,0.073227,0.043493,0.037420
6,cleveland_v2,0.025991,0.037908,0.029524
7,cmc,0.479466,0.501101,0.475940
8,dermatology,0.958125,0.960042,0.941425
9,glass,0.294955,0.150751,0.131206


In [14]:
results_df.filter(regex=("dataset|RF5-f1*"))

,dataset,RF5-f1-old-features,RF5-f1-old+new-features,RF5-f1-new-features
0,1czysty-cut,0.905533,0.936994,0.887043
1,2delikatne-cut,0.777951,0.753600,0.689131
2,3mocniej-cut,0.578375,0.574502,0.513390
3,4delikatne-bezover-cut,0.827701,0.824659,0.813733
4,balance-scale,0.609438,0.830439,0.806064
5,cleveland,0.284363,0.308840,0.257429
6,cleveland_v2,0.266816,0.304774,0.294584
7,cmc,0.514592,0.528700,0.489945
8,dermatology,0.969889,0.962956,0.951957
9,glass,0.688237,0.585527,0.491606


In [15]:
results_df.filter(regex=("dataset|RF5-gmean*"))

,dataset,RF5-gmean-old-features,RF5-gmean-old+new-features,RF5-gmean-new-features
0,1czysty-cut,0.902969,0.919901,0.879214
1,2delikatne-cut,0.725794,0.680167,0.587150
2,3mocniej-cut,0.416180,0.412393,0.266674
3,4delikatne-bezover-cut,0.815232,0.800310,0.773361
4,balance-scale,0.096721,0.820452,0.793688
5,cleveland,0.074887,0.071307,0.050629
6,cleveland_v2,0.025892,0.041312,0.030704
7,cmc,0.487395,0.502703,0.474851
8,dermatology,0.961607,0.956976,0.947861
9,glass,0.419342,0.175361,0.133640


In [16]:
results_df.filter(regex=("dataset|RF7-f1*"))

,dataset,RF7-f1-old-features,RF7-f1-old+new-features,RF7-f1-new-features
0,1czysty-cut,0.928631,0.955776,0.904779
1,2delikatne-cut,0.786406,0.773095,0.718577
2,3mocniej-cut,0.584513,0.595548,0.512915
3,4delikatne-bezover-cut,0.833611,0.824070,0.809200
4,balance-scale,0.608746,0.834250,0.787517
5,cleveland,0.296950,0.317990,0.257035
6,cleveland_v2,0.323781,0.335868,0.308302
7,cmc,0.541080,0.531180,0.493027
8,dermatology,0.969889,0.960594,0.940354
9,glass,0.704035,0.663647,0.490274


In [17]:
results_df.filter(regex=("dataset|RF7-gmean*"))

,dataset,RF7-gmean-old-features,RF7-gmean-old+new-features,RF7-gmean-new-features
0,1czysty-cut,0.924891,0.943420,0.899292
1,2delikatne-cut,0.747480,0.723168,0.651873
2,3mocniej-cut,0.452692,0.458436,0.276751
3,4delikatne-bezover-cut,0.828137,0.812660,0.792369
4,balance-scale,0.095293,0.809652,0.756226
5,cleveland,0.080578,0.087694,0.049438
6,cleveland_v2,0.053368,0.056487,0.046169
7,cmc,0.523951,0.512430,0.480539
8,dermatology,0.961607,0.956583,0.932151
9,glass,0.489533,0.375719,0.132573
